<a href="https://colab.research.google.com/github/mateofuertes/ImageToGcode/blob/OCR-Method-2/text_calssifier_ai_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install transformers torch datasets

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Load your data
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset

In [9]:
import pandas as pd
from datasets import Dataset

# Load data from CSV
csv_file_path = '/content/drive/MyDrive/dataset_names-address-position.csv'

# Read CSV file
df = pd.read_csv(csv_file_path, header=None, names=['text', 'label'])

# Display the DataFrame
print(df)

# Ensure there's enough data
print("DataFrame shape:", df.shape)
if df.shape[0] < 1:
    raise ValueError("Not enough data to create a training set.")

# Convert DataFrame to Dataset
train_dataset = Dataset.from_pandas(df)

# Print the first few rows of the Dataset
print("Train dataset:", train_dataset[0])


                                                   text     label
0                                      Stephanie Brooks      name
1                                Programmer, multimedia  position
2                    Surveyor, planning and development  position
3                             USNS Nguyen\nFPO AP 78051   address
4                               Mr. James Valentine DVM      name
...                                                 ...       ...
2995                                       Jamie Flores      name
2996                        Conservator, museum/gallery  position
2997                                    Jamie Hernandez      name
2998  268 Erika Crest Suite 849\nPort Toddberg, ME 5...   address
2999  054 Mcdonald Landing\nLake Catherinestad, WV 5...   address

[3000 rows x 2 columns]
DataFrame shape: (3000, 2)
Train dataset: {'text': 'Stephanie Brooks', 'label': 'name'}


In [11]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
from transformers import DistilBertTokenizerFast

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Load data from CSV
csv_file_path = '/content/drive/MyDrive/dataset_names-address-position.csv'
df = pd.read_csv(csv_file_path, header=None, names=['text', 'label'])

# Convert DataFrame to Dataset
train_dataset = Dataset.from_pandas(df)

# Collect all labels from the training dataset
training_labels = [example['label'] for example in train_dataset]

# Print collected labels for debugging
print("Collected labels from train_dataset:", training_labels)

# Fit the encoder with the training labels
label_encoder.fit(training_labels)

# Print label classes to verify the fitting
print("Classes found by LabelEncoder:", label_encoder.classes_)
print(len(label_encoder.classes_))

# Define label2id and id2label mappings
label2id = {label: i for i, label in enumerate(label_encoder.classes_)}
id2label = {i: label for label, i in label2id.items()}
print("label2id mapping:", label2id)
print("id2label mapping:", id2label)

# Function to encode labels
def encode_labels(example):
    labels = example['label']
    try:
        # Transform the labels to encoded values
        encoded_labels = label_encoder.transform([labels])
        return {'label': encoded_labels[0]}  # Return the first and only element
    except ValueError as e:
        print(f"Error during label transformation: {e}")
        return {'label': -1}  # Return -1 for invalid labels

# Apply encoding to train dataset
train_dataset = train_dataset.map(encode_labels)

# Print the resulting train dataset
print(train_dataset.shape)


Collected labels from train_dataset: ['name', 'position', 'position', 'address', 'name', 'name', 'position', 'position', 'name', 'name', 'position', 'position', 'name', 'name', 'position', 'address', 'address', 'address', 'position', 'position', 'position', 'name', 'address', 'position', 'address', 'name', 'address', 'name', 'name', 'name', 'address', 'name', 'address', 'name', 'address', 'name', 'address', 'position', 'address', 'position', 'position', 'name', 'name', 'name', 'position', 'position', 'name', 'position', 'name', 'name', 'address', 'position', 'name', 'position', 'address', 'name', 'name', 'address', 'name', 'name', 'address', 'position', 'name', 'position', 'position', 'position', 'name', 'address', 'position', 'name', 'position', 'position', 'address', 'address', 'address', 'address', 'address', 'address', 'name', 'address', 'name', 'position', 'name', 'name', 'address', 'name', 'address', 'address', 'address', 'address', 'address', 'name', 'address', 'name', 'address'

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

(3000, 2)


In [12]:
from transformers import DistilBertTokenizerFast
from torch.utils.data import DataLoader

# Load tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Tokenize data
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

# Tokenize the training dataset
train_dataset = train_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Create data loader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Print information about the DataLoader
print(f"Number of batches in train_loader: {len(train_loader)}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Number of batches in train_loader: 188


In [13]:
import torch
from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, TrainingArguments, Trainer
from transformers import AdamW, get_scheduler
from tqdm.auto import tqdm

# Load and train the model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label_encoder.classes_))

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
#     eval_dataset=test_dataset,
)

# Train the model
trainer.train()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,0.177400


TrainOutput(global_step=564, training_loss=0.15733442042709878, metrics={'train_runtime': 412.642, 'train_samples_per_second': 21.811, 'train_steps_per_second': 1.367, 'total_flos': 1192227849216000.0, 'train_loss': 0.15733442042709878, 'epoch': 3.0})

In [14]:
# Save the model
model.save_pretrained("business_card_text_classification")
tokenizer.save_pretrained("business_card_text_classification_tokenizer")

('business_card_text_classification_tokenizer/tokenizer_config.json',
 'business_card_text_classification_tokenizer/special_tokens_map.json',
 'business_card_text_classification_tokenizer/vocab.txt',
 'business_card_text_classification_tokenizer/added_tokens.json',
 'business_card_text_classification_tokenizer/tokenizer.json')